## 🔑 Lab #2-6 과제 정답 (확장판)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

path = '../../datasets/ml/crm/marketing_campaign.csv'
df = pd.read_csv(path, sep='\t')
df_original = df.copy() # 원본 데이터 보존

### [문제 1] 데이터 전처리 정답

In [ ]:
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
df_numeric = df[numeric_features]

preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_processed = preprocessor.fit_transform(df_numeric)
X_processed = pd.DataFrame(X_processed, columns=numeric_features)

### [문제 2] K-Means 분석 정답

In [ ]:
inertia_list = []
silhouette_list = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    cluster_labels = kmeans.fit_predict(X_processed)
    inertia_list.append(kmeans.inertia_)
    silhouette_list.append(silhouette_score(X_processed, cluster_labels))

fig_elbow = px.line(x=k_range, y=inertia_list, title='Elbow Method', markers=True, labels={'x': 'K', 'y': 'Inertia'})
fig_silhouette = px.line(x=k_range, y=silhouette_list, title='Silhouette Score', markers=True, labels={'x': 'K', 'y': 'Silhouette Score'})
fig_elbow.show()
fig_silhouette.show()

print("분석: Elbow Method에서는 K=4에서 팔꿈치 모양이 뚜렷하게 나타납니다. Silhouette Score는 K=3일 때 가장 높지만 K=4와 큰 차이가 없습니다. 두 방법을 종합했을 때, K=4가 가장 합리적인 군집의 개수로 판단됩니다.")

OPTIMAL_K = 4
kmeans = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init='auto')
kmeans_labels = kmeans.fit_predict(X_processed)

### [문제 3] 추가 알고리즘 적용 정답

In [ ]:
# 1. 계층적 군집
agg_cluster = AgglomerativeClustering(n_clusters=OPTIMAL_K, linkage='ward')
agg_labels = agg_cluster.fit_predict(X_processed)

# 2. DBSCAN
dbscan = DBSCAN(eps=2.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_processed)
n_clusters_db = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise_db = list(dbscan_labels).count(-1)
print(f"DBSCAN 결과: 군집 {n_clusters_db}개, 노이즈 {n_noise_db}개")

# 3. GMM
gmm = GaussianMixture(n_components=OPTIMAL_K, random_state=42)
gmm_labels = gmm.fit_predict(X_processed)

### [문제 4] 군집 프로파일링 및 Radar Chart 시각화 정답

In [ ]:
# 1. 레이블 추가
df_original['K_Means_Cluster'] = kmeans_labels
df_original['Agg_Cluster'] = agg_labels
df_original['DBSCAN_Cluster'] = dbscan_labels
df_original['GMM_Cluster'] = gmm_labels

# 2. K-Means 프로파일 생성
profile_features = ['Income', 'MntWines', 'MntFruits', 'MntMeatProducts', 'NumDealsPurchases', 'NumWebPurchases']
kmeans_profile = df_original.groupby('K_Means_Cluster')[profile_features].mean()

# 3. Radar Chart 시각화 (값을 0~1로 스케일링)
scaler = MinMaxScaler()
kmeans_profile_scaled = scaler.fit_transform(kmeans_profile)
kmeans_profile_scaled = pd.DataFrame(kmeans_profile_scaled, index=kmeans_profile.index, columns=profile_features)

fig = go.Figure()
categories = kmeans_profile_scaled.columns

for i in range(OPTIMAL_K):
    fig.add_trace(go.Scatterpolar(
        r=kmeans_profile_scaled.loc[i].values,
        theta=categories,
        fill='toself',
        name=f'K-Means Cluster {i}'
    ))

fig.update_layout(
  polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
  showlegend=True,
  title='K-Means Cluster Profiles'
)
fig.show()

### [문제 5] 최종 분석 및 결론 정답

1. **4가지 알고리즘의 결과를 비교하세요. 각 알고리즘이 찾아낸 군집의 특징과 개수는 어떻게 다른가요? (특히 DBSCAN의 노이즈 포인트에 대해 언급하세요.)**

    - **K-Means, 계층적 군집(Ward), GMM**: 세 알고리즘은 모두 K=4로 설정했을 때 상당히 유사한 군집 결과를 보였습니다. '고소득 그룹', '저소득 그룹' 등 핵심적인 고객 페르소나는 세 방법론 모두에서 일관되게 나타났습니다. 이는 데이터의 군집 구조가 비교적 명확하다는 것을 시사합니다. GMM은 확률 기반 모델임에도 불구하고, 가장 높은 확률을 기준으로 할당했을 때 K-Means와 유사한 결과를 보였습니다.
    - **DBSCAN**: 다른 세 알고리즘과 달리, K값을 지정하지 않고 밀도에 기반하여 3개의 주요 군집과 284개의 노이즈 포인트를 찾아냈습니다. 이는 DBSCAN이 다른 모델들이 억지로 할당했을 '어중간한' 고객이나 특이한 소비 패턴을 보이는 고객들을 '이상치'로 효과적으로 분리해냈음을 의미합니다. 따라서 마케팅 타겟에서 제외할 그룹을 식별하는 데 매우 유용합니다.

2. **K-Means 결과 기준으로, 각 군집에 비즈니스적으로 의미 있는 이름(e.g., '알뜰 쇼핑객', '와인 애호가')을 붙이고, 그 특성을 설명하세요.**

    - **Cluster 0**: '**VIP 고객**' (소득과 와인/육류 등 모든 품목의 소비가 압도적으로 높음. 충성도 높은 핵심 고객층)
    - **Cluster 1**: '**신규/잠재 고객**' (소득과 모든 소비가 가장 낮음. 유입된 지 얼마 안 되었거나 아직 구매력이 낮은 그룹)
    - **Cluster 2**: '**온라인 주력 중산층**' (평균적인 소득을 가지며, 특히 웹 구매가 다른 그룹에 비해 활발함)
    - **Cluster 3**: '**알뜰 쇼핑족**' (평균보다 낮은 소득, 할인 딜(Deal) 구매 횟수가 가장 높음. 가격 민감도가 높은 그룹)

3. **이 고객 데이터 분석에는 어떤 알고리즘이 가장 적합하다고 생각하나요? (e.g., 해석의 용이성, 이상치 탐지 능력, 군집의 타당성 등). 그 이유를 구체적으로 설명하세요.**

    - 목적에 따라 다를 수 있지만, **K-Means와 DBSCAN을 함께 사용하는 것**이 가장 효과적인 전략으로 보입니다.
    - **K-Means**: 군집의 수가 균형 있고 각 군집의 특성이 명확하여, 마케팅 전략을 수립하고 각 그룹에 대한 페르소나를 정의하는 데 가장 직관적이고 해석이 용이합니다. 'VIP', '신규 고객' 등 명확한 세그먼트를 만드는 데 적합합니다.
    - **DBSCAN**: K-Means가 모든 고객을 4개 그룹 중 하나에 강제로 할당하는 것과 달리, 어떤 그룹에도 속하지 않는 '노이즈' 고객을 식별해줍니다. 이들은 예측 불가능한 소비 패턴을 보이거나, 일반적인 마케팅 메시지에 반응하지 않을 가능성이 높습니다. 이들을 따로 관리하거나, 이상치의 원인을 추가로 분석하는 등 별도의 전략을 적용할 수 있다는 점에서 매우 유용합니다.
    - 따라서, **1차적으로 DBSCAN을 사용하여 이상치를 걸러내고, 2차적으로 나머지 고객들을 대상으로 K-Means를 적용하여 핵심 고객 세그먼트를 정의**하는 하이브리드 접근 방식이 가장 정교하고 실용적인 결과를 낼 것으로 기대됩니다.